In [ ]:
#pip install beautifulsoup4

In [73]:
#conda install -c conda-forge folium

In [72]:
#conda install -c conda-forge geopy

## Scrape the Wikipedia page

In [82]:
import bs4 as bs
import urllib
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests 
from pandas.io.json import json_normalize

# Part 1

In [9]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = bs.BeautifulSoup(source,'html.parser')

In [12]:
#Canada_data

In [25]:
col_nm = ['Postalcode','Borough','Neighborhood']
cty_data = pd.DataFrame(columns = column_names)

In [28]:
col_name1 = ['Postalcode','Borough','Neighborhood']
cty_data = pd.DataFrame(columns = col_nm)

# loop through to find postcode, borough, neighborhood 
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    cty_data = cty_data.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
cty_data = cty_data[cty_data.Borough!='Not assigned']
cty_data = cty_data[cty_data.Borough!= 0]
cty_data.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,cty_data.shape[0]):
    if cty_data.iloc[i][2] == 'Not assigned':
        cty_data.iloc[i][2] = cty_data.iloc[i][1]
        i = i+1
                                 
df = cty_data.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [23]:
print(df2.shape)
df2.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Part 2

In [32]:
!wget -q -O 'toronto_data.csv' http://cocl.us/Geospatial_data

In [60]:
geo_df = pd.read_csv("toronto_data.csv", delimiter=",")
geo_df = geo_df.rename(columns={'Postal Code':'Postalcode'})
geo_df.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [61]:
mrg_df = pd.merge(df, geo_df, on='Postalcode')
mrg_df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3

In [89]:
address = 'Toronto, CA'

geo_loc = Nominatim(user_agent="toronto_explorer")
loc1 = geo_loc.geocode(address)
lat1 = location.latitude
long1 = location.longitude
print('The geograpical coordinate of Tornoto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tornoto City are 43.653963, -79.387207.


In [93]:
map1 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(mrg_df['Latitude'], mrg_df['Longitude'], mrg_df['Borough'], mrg_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map1)  
    
map1

In [78]:
toronto_Borough = neighborhoods_geos[neighborhoods_geos['Borough'].str.contains('Toronto',regex=False)].reset_index(drop=True)
toronto_Borough.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [84]:
map_tornoto_borough = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_Borough['Latitude'], toronto_Borough['Longitude'], toronto_Borough['Borough'], toronto_Borough['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tornoto_borough)  
    
map_tornoto_borough

In [80]:
kclusters = 5
toronto_clustering = toronto_Borough.drop(['Postalcode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = kclusters,random_state=0).fit(toronto_clustering)
kmeans.labels_
toronto_Borough.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_Borough.head()

,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,0,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,0,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,0,M4M,East Toronto,Studio District,43.659526,-79.340923
4,1,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [83]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
kclusters = 5
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_Borough['Latitude'], toronto_Borough['Longitude'], toronto_Borough['Borough'], toronto_Borough['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters